In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
# following Chollet's example at https://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb

In [2]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 250, 250, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 122, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 122, 122, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Step 1 - train new classifier with frozen base model

In [3]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
resnet.trainable = False
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 250, 250, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 122, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 122, 122, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, Dropout
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               33554688  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 57,144,970
Trainable params: 33,557,258
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
from load_data import load_data_gen

data_gen_params = {
    "rotation_range": 40,
    "width_shift_range": 0.2,
    "height_shift_range": 0.2,
    "shear_range": 0.2,
    "zoom_range": 0.2,
    "horizontal_flip": True,
}
(train_data, val_data, test_data) = load_data_gen(task=2, img_dims=244, **data_gen_params)

Found 11209 images belonging to 10 classes.
Found 1245 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [6]:
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  print(labels_batch[1])
  break

(32, 244, 244, 3)
(32, 10)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [7]:
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
)

Epoch 1/30
351/351 [==============================] - 226s 617ms/step - loss: 11.5509 - accuracy: 0.1175 - val_loss: 2.5463 - val_accuracy: 0.1205

Step 2 - unfreeze res5c block and retrain final layers and added classifier

In [17]:
resnet.trainable = True

res5c = 0
trainable_layer_count = 0
for layer in resnet.layers:
    if layer.name[0:5] == "conv5":
        res5c = 1
    if not res5c:
        layer.trainable = False
    if layer.trainable:
        trainable_layer_count += 1
        print(layer.name)

print(f"{trainable_layer_count} trainable layers")
resnet.summary()

conv5_block1_1_conv
conv5_block1_1_bn
conv5_block1_1_relu
conv5_block1_2_conv
conv5_block1_2_bn
conv5_block1_2_relu
conv5_block1_0_conv
conv5_block1_3_conv
conv5_block1_0_bn
conv5_block1_3_bn
conv5_block1_add
conv5_block1_out
conv5_block2_1_conv
conv5_block2_1_bn
conv5_block2_1_relu
conv5_block2_2_conv
conv5_block2_2_bn
conv5_block2_2_relu
conv5_block2_3_conv
conv5_block2_3_bn
conv5_block2_add
conv5_block2_out
conv5_block3_1_conv
conv5_block3_1_bn
conv5_block3_1_relu
conv5_block3_2_conv
conv5_block3_2_bn
conv5_block3_2_relu
conv5_block3_3_conv
conv5_block3_3_bn
conv5_block3_add
conv5_block3_out
32 trainable layers
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
_______________________________________________________________

In [ ]:
# adapted from Paul Whelan's course notes
def smoothing(data, f=0.8):
    smoothed = []
    for d in data:
        smoothed_point = (smoothed[-1] * f + d * (1 - f)) if smoothed else d
        smoothed.append(smoothed_point)
    return smoothed